In [ ]:
import pandas as pd
import numpy as np
panel = pd.read_csv("mei/core_macro.csv")
panel.set_index('date', inplace=True)
panel.index = pd.to_datetime(panel.index)
panel.drop("Currency_conversions_usd", axis=1, inplace=True)

from dataloader import MacroDataLoader

col = ["Total_industry", "CPI_all_items", "Harmonised_unemployment", "CPI_core",
        "Exports", "Imports", "Share_prices",
        "IR_10months", "Consumer_confidence"]
cfg = {
    'date_col': 'date',
    'country_col': 'country',
    'target_col': col,
    'features': col
}
loader = MacroDataLoader(panel, cfg)

from model import *
import numpy as np

models_dict = {
    'RNN -5': make_builder(RNNModel, config_rnn, weight_decay=1e-5, verbose=True, epochs=300),
    'RNN -4': make_builder(RNNModel, config_rnn, weight_decay=1e-4, verbose=True, epochs=300),
    'RNN -3': make_builder(RNNModel, config_rnn, weight_decay=1e-3, verbose=True, epochs=300),

}

from evaluator_monitoring import ContinuousSplitEvaluator

backtest_dates = pd.date_range(start='2013-01', periods=12*6, freq='ME')

evaluator = ContinuousSplitEvaluator(
    dataloader=loader,
    models_builders=models_dict,
    horizon=1,
    verbose=True,
)

results = evaluator.run(
    backtest_dates=backtest_dates,
    n_splits = 1,
)

evaluator.plot_learning_curves(results)

[MacroDataLoader] Initialized: 8 entities, 3283 obs, 9 features.

MONITORING MODEL: RNN -5

>> Split 1/1 : 2013-01-31 -> 2018-12-31 (72 obs)
weight decay = 1e-05
